# IR Lab WiSe 2024/2025: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will work in the MS MARCO scenario (retrieving passages of web documents). This Jupyter notebook serves as retrieval system and makes a submission to TIRA.

An overview of all corpora that we use in the course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset ids with which you can load the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the 2019 and 2020 TREC Deep Learning tracks on the MS MARCO v1 passage dataset. You can use this dataset to develop your system.
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (work in progress): A subsample of the 2024 TREC RAG track on the MS MARCO v2.1 passage dataset. You can use this dataset to develop your system.
- `ir-lab-wise-2024/ms-marco-rag-20241203-test`: (Not ready yet). The test corpus that we all developed together throughout the course on the MS MARCO v2.1 passage dataset. This dataset is the final test dataset, i.e., evaluation scores become available only after the submission deadline.


### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [ ]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0'

In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [4]:
# Create a REST client to the TIRA platform to load datasets and submit runs.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset

In [7]:
# The dataset: a subsample of the 2019 and 2020 MS MARCO TREC Deep Learning Track
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')


### Step 3: Build an Index


The type of the index object that we build is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [8]:
indexer = pt.IterDictIndexer("./index", meta={'docno': 50, 'text': 4096}, overwrite=True)
index = indexer.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 25667/68261 [00:11<00:05, 7752.61it/s]

09:50:42.446 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:15<00:00, 4312.86it/s] 


09:50:49.742 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


### Step 4: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [9]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

### Step 5: Create the Run


In [10]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia


In [11]:
print('Now we do the retrieval...')
run = bm25(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Persist and Upload the Run for Subsequent Evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated for which we upload it to TIRA.

In [12]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs', upload_to_tira=pt_dataset)

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt.gz".
Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/65591c62-80ea-4700-81ec-b5c6ede07707
